Project plan:

Evcxr not ideal for complex DS project due to slow startup for dependencies (data wrangling could be a challenge), so will use Polars-Python for data cleaning instead. However, Rust is a static programming language that has a lot of advantages. To reach wider audience, using Evcxr_jupyter to demonstrate Rust code interactively in data viz.

Part 1: Use Polars-Python for data wrangling

Part 2: Import wrangled dataset into Evcxr-Rust for data visualisation in Plotly.rs

In [1]:
#pip install --upgrade polars

In [2]:
import polars as pl

In [3]:
pl.show_versions()

---Version info---
Polars: 0.15.13
Index type: UInt32
Platform: macOS-10.15.7-x86_64-i386-64bit
Python: 3.10.8 (main, Oct 13 2022, 10:19:13) [Clang 12.0.0 (clang-1200.0.32.29)]
---Optional dependencies---
pyarrow: 10.0.1
pandas: 1.5.1
numpy: 1.23.4
fsspec: <not installed>
connectorx: <not installed>
xlsx2csv: 0.8
matplotlib: 3.6.2


Mention the Pillbox link was retired from 28th January 2021, but was available for education/research purposes (not for pill identifications, alternative links to e.g. DailyMed would be more appropriate).

When importing pillbox.csv file initially, an error message came up with "...Could not parse '10.16' as dtype Int64 at column 7...". One way to get around this was to add "ignore_errors" to bypass this error first in order to load the dataset. This error could be fixed when checking and converting data types for columns in later steps.

#### **Full dataframe**

In [4]:
df = pl.read_csv("pillbox.csv", ignore_errors = True)
df

ID,Enabled?,created at,updated at,spp,setid,splsize,pillbox_size,splshape,splshape_text,pillbox_shape_text,splscore,pillbox_score,splimprint,pillbox_imprint,splcolor,splcolor_text,pillbox_color_text,spl_strength,spl_ingredients,spl_inactive_ing,source,rxtty,rxstring,rxcui,RxNorm Update time,product_code,part_num,part_medicine_name,ndc9,ndc_labeler_code,ndc_product_code,medicine_name,marketing_act_code,effective_time,file_name,equal_product_code,dosage_form,document_type,dea_schedule_code,dea_schedule_name,author_type,author,approval_code,image_source,splimage,has_image,epc_match,version_number,laberer_code,application_number,updated,stale,new,Pillbox Value
i64,bool,str,str,str,str,i64,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,i64,str,i64,i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,bool,i64,i64,str,str,bool,bool,bool,bool
41846,true,"""10/17/2017 05:...","""10/02/2020 05:...","""471fa2f1-73a0-...","""471fa2f1-73a0-...",16,null,"""C48336""","""CAPSULE""",null,1,null,"""5892;V""",null,"""C48328""","""PINK""",null,"""TEMAZEPAM 15 m...","""TEMAZEPAM[TEMA...","""SILICON DIOXID...","""HRX""",null,null,null,"""10/02/2020 04:...","""0603-5892""",0,null,6035892,603,5892,"""Temazepam""","""completed""",20160406,"""d912ca54-6569-...",null,"""C25158""",null,"""C48677""","""CIV""","""LABELER""","""Qualitest Phar...","""C73584""",null,null,false,null,5,null,null,true,false,false,false
8100,true,"""10/17/2017 05:...","""10/02/2020 04:...","""116e13c1-ac50-...","""116e13c1-ac50-...",10,null,"""C48348""","""ROUND""",null,1,null,"""I2""",null,"""C48331""","""ORANGE""",null,"""IBUPROFEN 200 ...","""IBUPROFEN[IBUP...","""SILICON DIOXID...","""HOTC""","""SCD""","""ibuprofen 200 ...",310965,"""10/02/2020 03:...","""59779-074""",0,null,597790074,59779,74,"""ibuprofen""","""active""",20191120,"""55de9f94-89b2-...",null,"""C42931""","""34390-5""",null,null,"""LABELER""","""CVS Pharmacy""","""C73584""",null,null,false,null,4,null,null,true,false,false,false
5258,true,"""10/17/2017 05:...","""10/17/2017 05:...","""827ce261-307b-...","""827ce261-307b-...",7,null,"""C48346""","""PENTAGON (5 SI...",null,1,2,"""par;129""",null,"""C48329""","""GREEN""",null,"""DEXAMETHASONE ...","""DEXAMETHASONE[...","""ANHYDROUS LACT...","""HRX""",null,"""Dexamethasone ...",197583,null,"""49884-129""",0,null,498840129,49884,129,"""Dexamethasone""","""active""",20120516,"""85a9eebb-be74-...",null,"""C42998""","""34391-3""",null,null,"""LABELER""","""Par Pharmaceut...","""C73584""","""NLM""","""498840129""",true,null,4,null,null,false,true,false,true
21271,true,"""10/17/2017 05:...","""10/02/2020 05:...","""f7f1c99e-1a67-...","""f7f1c99e-1a67-...",11,null,"""C48348""","""ROUND""",null,2,null,"""LL""",null,"""C48325""","""WHITE""",null,"""Nickel Sulfate...","""Nickel Sulfate...",null,"""HOMEO""",null,null,null,"""10/02/2020 04:...","""61480-137""",0,null,614800137,61480,137,"""Acunol""","""active""",20190909,"""029eaf64-e66f-...",null,"""C42998""","""34391-3""",null,null,"""LABELER""","""PLYMOUTH HEALT...","""C73614""",null,null,false,null,8,null,null,true,false,false,false
77050,true,"""09/20/2019 09:...","""10/02/2020 05:...","""ecb28fcb-f0d1-...","""ecb28fcb-f0d1-...",6,null,"""C48348""","""ROUND""",null,1,null,"""L;524""",null,"""C48325""","""WHITE""",null,"""CLONAZEPAM 0.2...","""CLONAZEPAM[CLO...","""SORBITOL;ASPAR...","""HRX""","""SCD""","""clonazepam 0.2...",349195,"""10/02/2020 04:...","""62332-365""",0,null,623320365,62332,365,"""CLONAZEPAM""","""active""",20190701,"""ba5120ce-ed74-...",null,"""C42999""",null,"""C48677""","""CIV""","""LABELER""","""Alembic Pharma...","""C73584""",null,null,false,null,3,null,null,true,false,false,false
76916,true,"""09/20/2019 09:...","""10/02/2020 05:...","""442e41da-24c2-...","""442e41da-24c2-...",9,null,"""C48348""","""ROUND""",null,1,null,"""LU;V06""",null,"""C48325""","""WHITE""",null,"""SILDENAFIL CIT...","""SILDENAFIL CIT...","""ANHYDROUS DIBA...","""HRX""","""SCD""","""s

#### **Data wrangling**

The [Pillbox URL link from NLM](https://datadiscovery.nlm.nih.gov/Drugs-and-Chemicals/Pillbox-retired-January-28-2021-/crzr-uvwg) provided a list of column information for users. Here, we could keep what were needed for our data visualisation in part 2 of the project by extracting a subset of data from the full dataset.

In [5]:
print(df.glimpse())

Rows: 83925
Columns: 55
$ ID                   <Int64> 41846, 8100, 5258, 21271, 77050, 76916, 20016, 67902, 75997, 1288
$ Enabled?           <Boolean> True, True, True, True, True, True, True, True, True, True
$ created at            <Utf8> 10/17/2017 05:32:23 PM, 10/17/2017 05:29:56 PM, 10/17/2017 05:29:44 PM, 10/17/2017 05:30:52 PM, 09/20/2019 09:10:47 PM, 09/20/2019 09:10:41 PM, 10/17/2017 05:30:47 PM, 06/27/2019 10:45:17 PM, 09/20/2019 09:09:57 PM, 10/17/2017 05:29:25 PM
$ updated at            <Utf8> 10/02/2020 05:14:07 PM, 10/02/2020 04:59:28 PM, 10/17/2017 05:29:44 PM, 10/02/2020 05:10:28 PM, 10/02/2020 05:14:51 PM, 10/02/2020 05:15:50 PM, 10/02/2020 05:12:38 PM, 10/02/2020 05:04:25 PM, 10/02/2020 05:12:06 PM, 10/17/2017 05:29:25 PM
$ spp                   <Utf8> 471fa2f1-73a0-49be-89f3-d3e2cfdaeca0-0603-5892-0, 116e13c1-ac50-400f-880f-5779f0155b96-59779-074-0, 827ce261-307b-4398-8993-333c08e601fe-49884-129-0, f7f1c99e-1a67-4b34-b1f4-0ac38b9d8006-61480-137-0, ecb28fcb-f0d1-4558

Since the plan was for a data visualisation in Rust-evcxr eventually, a relatively simple dataset would be extracted for these pills data. Therefore, only certain columns would be selected for this purpose.

In [6]:
df_med = df.select([# shapes of medicines
                    "splshape_text", 
                    # colours of medicines
                    "splcolor_text",
                    # strengths of medicines
                    "spl_strength", 
                    # inactive ingredients/excipients in medicines  
                    "spl_inactive_ing",
                    # dosage forms of medicines e.g. capsules or tablets etc.
                    "dosage_form"]
                  )
df_med

splshape_text,splcolor_text,spl_strength,spl_inactive_ing,dosage_form
str,str,str,str,str
"""CAPSULE""","""PINK""","""TEMAZEPAM 15 m...","""SILICON DIOXID...","""C25158"""
"""ROUND""","""ORANGE""","""IBUPROFEN 200 ...","""SILICON DIOXID...","""C42931"""
"""PENTAGON (5 SI...","""GREEN""","""DEXAMETHASONE ...","""ANHYDROUS LACT...","""C42998"""
"""ROUND""","""WHITE""","""Nickel Sulfate...",null,"""C42998"""
"""ROUND""","""WHITE""","""CLONAZEPAM 0.2...","""SORBITOL;ASPAR...","""C42999"""
"""ROUND""","""WHITE""","""SILDENAFIL CIT...","""ANHYDROUS DIBA...","""C42931"""
"""OVAL""","""YELLOW""","""RISPERIDONE 3 ...","""LACTOSE MONOHY...","""C42931"""
"""CAPSULE""","""BLUE""","""IBUPROFEN 200 ...","""FD&C BLUE NO. ...","""C42954"""
"""ROUND""","""WHITE""","""Iloperidone 12...","""silicon dioxid...","""C42998"""


#### **Web scraping**

In [7]:
# web scraping the FDA Dosage Forms URL link 
# https://www.fda.gov/industry/structured-product-labeling-resources/dosage-forms
# to be able to tell what codes correspond to which dosage forms

# Uncomment lines below to install libraries needed for web-scraping
#!pip install requests
#!pip install beautifulsoup4

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
url = "https://www.fda.gov/industry/structured-product-labeling-resources/dosage-forms"
data = requests.get(url)

In [10]:
soup = BeautifulSoup(data.content, "html.parser")

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-22737364-1">
  </script>
  <meta content="This web page has a list of dosage form terms and National Cancer Institute Thesaurus concept codes associated with those term for use in Structured Product Labeling (SPL) documents submitted to FDA." name="description"/>
  <meta content="Dosage Forms" name="dcterms.title"/>
  <meta content="Office of the Commissioner" name="dcterms.creator"/>
  <meta content="This web page has a list of dosage form terms and National Cancer Institut

#### **Transform web-scraped data into dataframe**

##### **Using Pandas dataframe library**

The original pandas.append() method was going to be deprecated in the future versions of Pandas. 

dosage_form = pd.DataFrame(columns = ["Dosage_form", "Code"])

for row in soup.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        dosage = col[0].text
        code = col[1].text
        dosage_form = dosage_form.append({"Dosage_form":dosage, "Code":code}, ignore_index = True)

dosage_form

This method might still work currently, however, we could use the pandas.concat() method as shown below.

In [12]:
# Using pd.concat() method

# Create an empty dictionary
dict = []

# Create a loop to iterate through the html tags from the soup (web-scraped html content)
# find all html tags that began with <tr>
for row in soup.find_all("tr"):
    # each column would hold the items under the html tags that began with <td>
    col = row.find_all("td")
    if (col != []): 
        # dosage form in column 1
        dosage = col[0].text
        # code in column 2
        code = col[1].text
        # Append each dosage form & code into the dict
        dict.append({"DosageForm": dosage, "dosage_form": code})

# Check if the loop was iterating through the html tags in the soup
# and also it was appending each dosage form & code into the dictionary (uncomment line below)
#print(dict)

# Create an empty dataframe with the column names wanted
dosage_form = pd.DataFrame(columns = ["DosageForm", "dosage_form"])

# Concatenate dosage_form dataframe & the dataframe converted from dict
df_new = pd.concat([dosage_form, pd.DataFrame.from_dict(dict)])

# Print the combined dataframe df_new
df_new

,DosageForm,dosage_form
0,AEROSOL,C42887
1,"AEROSOL, FOAM",C42888
2,"AEROSOL, METERED",C42960
3,"AEROSOL, POWDER",C42971
4,"AEROSOL, SPRAY",C42889
...,...,...
153,TAMPON,C47892
154,TAPE,C47897
155,TINCTURE,C43000
156,TROCHE,C43001


In [13]:
# Using pd.from_dict() method (a quicker way)

# Create an empty dictionary
dict = []

# Create a loop to iterate through the html tags from the soup (web-scraped html content)
# find all html tags that began with <tr>
for row in soup.find_all("tr"):
    # each column would hold the items under the html tags that began with <td>
    col = row.find_all("td")
    if (col != []): 
        # dosage form in column 1
        dosage = col[0].text
        # code in column 2
        code = col[1].text
        # Append each dosage form & code into the dict
        dict.append({"DosageForm": dosage, "dosage_form": code})

# Check if the loop was working to iterate through the html tags in the soup
# and also it was appending each dosage form & code into the dictionary (uncomment line below)
#print(dict)

# Convert the dictionary into a dataframe
df_new = pd.DataFrame.from_dict(dict)

# Print the dataframe df_new
df_new

,DosageForm,dosage_form
0,AEROSOL,C42887
1,"AEROSOL, FOAM",C42888
2,"AEROSOL, METERED",C42960
3,"AEROSOL, POWDER",C42971
4,"AEROSOL, SPRAY",C42889
...,...,...
153,TAMPON,C47892
154,TAPE,C47897
155,TINCTURE,C43000
156,TROCHE,C43001


##### **Using Polars dataframe library**

In [14]:
# Using Polars df library

# Create an empty dictionary
dict = []

# Create a loop to iterate through the html tags from the soup (web-scraped html content)
# find all html tags that began with <tr>
for row in soup.find_all("tr"):
    # each column would hold the items under the html tags that began with <td>
    col = row.find_all("td")
    if (col != []): 
        # dosage form in column 1
        dosage = col[0].text
        # code in column 2
        code = col[1].text
        # Append each dosage form & code into the dict
        dict.append({"DosageForm": dosage, "dosage_form": code})

# Check if the loop was iterating through the html tags in the soup
# and that it was also appending each dosage form & code into the dictionary (uncomment line below)
#print(dict)

# Convert dictionary to dataframe
new_df = pl.from_dicts(dict)
new_df

DosageForm,dosage_form
str,str
"""AEROSOL""","""C42887"""
"""AEROSOL, FOAM""","""C42888"""
"""AEROSOL, METER...","""C42960"""
"""AEROSOL, POWDE...","""C42971"""
"""AEROSOL, SPRAY...","""C42889"""
"""BAR, CHEWABLE""","""C42892"""
"""BEAD""","""C42890"""
"""CAPSULE""","""C25158"""
"""CAPSULE, COATE...","""C42895"""


In [16]:
df_final = df_med.join(new_df, on = "dosage_form")
df_final

splshape_text,splcolor_text,spl_strength,spl_inactive_ing,dosage_form,DosageForm
str,str,str,str,str,str
"""CAPSULE""","""PINK""","""TEMAZEPAM 15 m...","""SILICON DIOXID...","""C25158""","""CAPSULE"""
"""ROUND""","""ORANGE""","""IBUPROFEN 200 ...","""SILICON DIOXID...","""C42931""","""TABLET, FILM C..."
"""PENTAGON (5 SI...","""GREEN""","""DEXAMETHASONE ...","""ANHYDROUS LACT...","""C42998""","""TABLET"""
"""ROUND""","""WHITE""","""Nickel Sulfate...",null,"""C42998""","""TABLET"""
"""ROUND""","""WHITE""","""CLONAZEPAM 0.2...","""SORBITOL;ASPAR...","""C42999""","""TABLET, ORALLY..."
"""ROUND""","""WHITE""","""SILDENAFIL CIT...","""ANHYDROUS DIBA...","""C42931""","""TABLET, FILM C..."
"""OVAL""","""YELLOW""","""RISPERIDONE 3 ...","""LACTOSE MONOHY...","""C42931""","""TABLET, FILM C..."
"""CAPSULE""","""BLUE""","""IBUPROFEN 200 ...","""FD&C BLUE NO. ...","""C42954""","""CAPSULE, LIQUI..."
"""ROUND""","""WHITE""","""Iloperidone 12...","""silicon dioxid...","""C42998""","""TABLET"""
